In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numba import jit

# 1a

In [9]:
def getAcc(pos : np.ndarray, mass : np.ndarray, G : float, softening : float):
    """
    Calculate the acceleration on each particle due to Newton's Law
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    # positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]
    
    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)**(-1.5)
    
    ax = G * (dx * inv_r3) @ mass
    ay = G * (dy * inv_r3) @ mass
    az = G * (dz * inv_r3) @ mass
    
    # pack together the acceleration components
    a = np.hstack((ax,ay,az))

    return a

def leapfrog(pos: np.ndarray, vec: np.ndarray, mass: float,G: float, softening: float, dt: float):

    # leapfrog for N-body
    vec = vec + 0.5*dt*getAcc(pos, mass, G, softening)
    pos = pos+ vec*dt
    vec = vec+ 0.5*dt*getAcc(pos, mass, G, softening)

    return pos, vec

In [10]:
# Data and constants
plummer_data_1a = pd.read_csv('plummer_regular.csv')
r = plummer_data_1a.loc[:,['r_x', 'r_y', 'r_z']]
r = r.to_numpy()
v = plummer_data_1a.loc[:,['v_x', 'v_y', 'v_z']]
v = v.to_numpy()
mass = np.full((10000,1), 2)

# simulation time
T = 10
dt = 1
step = T/dt

# result container
pos_out = []
vec_out = []

In [11]:
# x = np.array([3,27,5.3]).reshape(3,1)
# print(x.T-x)

In [ ]:
for i in range(int(step)):
    r, v = leapfrog(r, v, mass, G = 1, softening = 5, dt = dt)
    pos_out.append(r)
    vec_out.append(v)

pos_out = np.array(pos_out)
vec_out = np.array(vec_out)